In [ ]:
import numpy as np
import pandas as pd 
from StructuraBreaks import *
# price = pd.read_csv("logprice.csv")


In [ ]:
price.index = price.iloc[:,0]
price = pd.DataFrame(price.iloc[:,1])

In [ ]:
ADF(price,100,"nc",40)